# Assignment 2

In this assigment, we will work with the *Adult* data set. Please download the data from the [UCI Machine Learning Repository](https://archive.ics.uci.edu/dataset/2/adult). Extract the data files into the subdirectory: `../05_src/data/adult/` (relative to `./05_src/`).

# Load the data

Assuming that the files `adult.data` and `adult.test` are in `../05_src/data/adult/`, then you can use the code below to load them.

In [11]:
# Define a function to find the file path for adult.data
import os

def find_adult_data():
    """
    Function to find the file '../05_src/data/adult/adult.data' using os.walk
    starting from the grandparents folder (two levels up from the current working directory).

    Returns:
    - str: The full file path if found, or None if not found.
    """
    target_file_path = None
    current_dir = os.getcwd()  # Get the current working directory

    # Navigate two levels up from the current directory
    grandparents_dir = os.path.abspath(os.path.join(current_dir, '../../'))

    # Walk through the directory tree starting from the grandparents directory
    for dirpath, dirnames, filenames in os.walk(grandparents_dir):
        # Check if '05_src/data/adult/adult.data' exists in the current directory structure
        check_path = os.path.join(dirpath, '05_src/data/adult/adult.data')
        if os.path.exists(check_path):
            target_file_path = check_path
            break
    
    return target_file_path

In [12]:
import pandas as pd
columns = [
    'age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status',
    'occupation', 'relationship', 'race', 'sex', 'capital-gain', 'capital-loss', 'hours-per-week',
    'native-country', 'income'
]
adult_dt = (pd.read_csv(find_adult_data(), header = None, names = columns)
              .assign(income = lambda x: (x.income.str.strip() == '>50K')*1))


In [13]:
adult_dt

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,0
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,0
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,0
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,0
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,27,Private,257302,Assoc-acdm,12,Married-civ-spouse,Tech-support,Wife,White,Female,0,0,38,United-States,0
32557,40,Private,154374,HS-grad,9,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,40,United-States,1
32558,58,Private,151910,HS-grad,9,Widowed,Adm-clerical,Unmarried,White,Female,0,0,40,United-States,0
32559,22,Private,201490,HS-grad,9,Never-married,Adm-clerical,Own-child,White,Male,0,0,20,United-States,0


# Get X and Y

Create the features data frame and target data:

+ Create a dataframe `X` that holds the features (all columns that are not `income`).
+ Create a dataframe `Y` that holds the target data (`income`).
+ From `X` and `Y`, obtain the training and testing data sets:

    - Use a train-test split of 70-30%. 
    - Set the random state of the splitting function to 42.

In [15]:
from sklearn.model_selection import train_test_split
# Define X and Y
X = adult_dt.drop(columns=['income'])
Y = adult_dt['income']

# Train-test split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=42)


## Random States

Please comment: 

+ What is the [random state](https://scikit-learn.org/stable/glossary.html#term-random_state) of the [splitting function](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html)? 
+ Why is it [useful](https://en.wikipedia.org/wiki/Reproducibility)?

*The random state parameter in the train-test splitting function (train_test_split) is useful for ensuring reproducibility. When you set a specific random state (e.g., random_state=42), it ensures that every time you run the splitting function with the same data and parameters, you get the same split of data into training and testing sets. This reproducibility is crucial for debugging, validation, and sharing results because it eliminates randomness in the data split process.*

# Preprocessing

Create a [Column Transformer](https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html) that treats the features as follows:

- Numerical variables

    * Apply [KNN-based imputation for completing missing values](https://scikit-learn.org/stable/modules/generated/sklearn.impute.KNNImputer.html):
        
        + Consider the 7 nearest neighbours.
        + Weight each neighbour by the inverse of its distance, causing closer neigbours to have more influence than more distant ones.
    * [Scale features using statistics that are robust to outliers](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.RobustScaler.html#sklearn.preprocessing.RobustScaler).

- Categorical variables: 
    
    * Apply a [simple imputation strategy](https://scikit-learn.org/stable/modules/generated/sklearn.impute.SimpleImputer.html#sklearn.impute.SimpleImputer):

        + Use the most frequent value to complete missing values, also called the *mode*.

    * Apply [one-hot encoding](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html):
        
        + Handle unknown labels if they exist.
        + Drop one column for binary variables.
    
    
The column transformer should look like this:

![](./images/assignment_2__column_transformer.png)

In [19]:
from sklearn.pipeline import Pipeline
from sklearn.impute import KNNImputer, SimpleImputer
from sklearn.preprocessing import RobustScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
# 1. Define numeric, categorical, and datetime features
# Avoid to get binary categorical features as numeric ones

numeric_features = []
categorical_features = []
datetime_features = []

for col in X.columns:
    if X[col].dtype in ['int64', 'float64']:
        # Exclude binary columns
        unique_values = X[col].dropna().unique()
        if len(unique_values) > 2:  # More than 2 unique values (not binary)
            numeric_features.append(col)
        else:  # Binary columns
            categorical_features.append(col)
    elif X[col].dtype == 'object' or X[col].dtype.name == 'category':
        categorical_features.append(col)
    elif X[col].dtype == 'datetime64[ns]':
        datetime_features.append(col)

# 2. Define non binary categorical columns and binary categorical columns
# Get non binary categorical features only for one hot encoding
binary_categorical_features = []
non_binary_categorical_features = []
for col in categorical_features:
    unique_values = X[col].dropna().unique()
    if set(unique_values) <= {0, 1}:
        binary_categorical_features.append(col)
    else:
        non_binary_categorical_features.append(col)

# 3. Handle missing values and scale numerical features
numeric_transformer = Pipeline(steps=[
    ('imputer', KNNImputer(n_neighbors=7, weights='distance')),
    ('scaler', RobustScaler())
])

# 4. Combine imputer and onehot for non-binary categorical features
# There are only non-binary categorical features in this dataset except for target variable 'income'
# therefore categorical_features = non_binary_categorical_features
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', drop='first'))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num_transforms', numeric_transformer, numeric_features),
        ('cat_transforms', categorical_transformer, non_binary_categorical_features)
    ])
preprocessor

ColumnTransformer(transformers=[('num_transforms',
                                 Pipeline(steps=[('imputer',
                                                  KNNImputer(n_neighbors=7,
                                                             weights='distance')),
                                                 ('scaler', RobustScaler())]),
                                 ['age', 'fnlwgt', 'education-num',
                                  'capital-gain', 'capital-loss',
                                  'hours-per-week']),
                                ('cat_transforms',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('onehot',
                                                  OneHotEncoder(drop='first',
                                                                handle_unknown='ignore'))]),
                                 ['workclass', 'education', 'marital-status',
                                  'occupation', 'relationship', 'race', 'sex',
                                  'native-country'])])

## Model Pipeline

Create a [model pipeline](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html): 

+ Add a step labelled `preprocessing` and assign the Column Transformer from the previous section.
+ Add a step labelled `classifier` and assign a [`RandomForestClassifier()`](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html) to it.

The pipeline looks like this:

![](./images/assignment_2__pipeline.png)

In [21]:
from sklearn.ensemble import RandomForestClassifier
model_pipeline = Pipeline(steps=[
    ('preprocessing', preprocessor),
    ('classifier', RandomForestClassifier())
])
model_pipeline

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('num_transforms',
                                                  Pipeline(steps=[('imputer',
                                                                   KNNImputer(n_neighbors=7,
                                                                              weights='distance')),
                                                                  ('scaler',
                                                                   RobustScaler())]),
                                                  ['age', 'fnlwgt',
                                                   'education-num',
                                                   'capital-gain',
                                                   'capital-loss',
                                                   'hours-per-week']),
                                                 ('cat_transforms',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehot',
                                                                   OneHotEncoder(drop='first',
                                                                                 handle_unknown='ignore'))]),
                                                  ['workclass', 'education',
                                                   'marital-status',
                                                   'occupation', 'relationship',
                                                   'race', 'sex',
                                                   'native-country'])])),
                ('classifier', RandomForestClassifier())])

# Cross-Validation

Evaluate the model pipeline using [`cross_validate()`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_validate.html):

+ Measure the following [preformance metrics](https://scikit-learn.org/stable/modules/model_evaluation.html#common-cases-predefined-values): negative log loss, ROC AUC, accuracy, and balanced accuracy.
+ Report the training and validation results. 
+ Use five folds.


In [23]:
from sklearn.metrics import make_scorer, roc_auc_score, accuracy_score, balanced_accuracy_score, log_loss
from sklearn.model_selection import cross_validate
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=UserWarning)

scoring = {
    'neg_log_loss': make_scorer(log_loss, greater_is_better=False, needs_proba=True),
    'roc_auc': make_scorer(roc_auc_score),
    'accuracy': make_scorer(accuracy_score),
    'balanced_accuracy': make_scorer(balanced_accuracy_score)
}
cv_results = cross_validate(model_pipeline, X_train, Y_train, scoring=scoring, cv=5)


Display the fold-level results as a pandas data frame and sorted by negative log loss of the test (validation) set.

In [24]:
cv_results_df = pd.DataFrame(cv_results)
cv_results_df_sorted = cv_results_df.sort_values(by='test_neg_log_loss', ascending=True)
cv_results_df_sorted

,fit_time,score_time,test_neg_log_loss,test_roc_auc,test_accuracy,test_balanced_accuracy
2,11.691694,0.278682,-0.380287,0.777519,0.854980,0.777519
4,11.875321,0.273446,-0.379737,0.774261,0.855638,0.774261
1,11.831842,0.281428,-0.358813,0.765623,0.845580,0.765623
0,11.678357,0.290462,-0.348664,0.772803,0.850406,0.772803
3,11.907641,0.283309,-0.341492,0.780759,0.858491,0.780759


Calculate the mean of each metric. 

In [25]:
mean_metrics = cv_results_df.mean()
mean_metrics

fit_time                  11.796971
score_time                 0.281465
test_neg_log_loss         -0.361799
test_roc_auc               0.774193
test_accuracy              0.853019
test_balanced_accuracy     0.774193
dtype: float64

Calculate the same performance metrics (negative log loss, ROC AUC, accuracy, and balanced accuracy) using the testing data `X_test` and `Y_test`. Display results as a dictionary.

*Tip*: both, `roc_auc()` and `neg_log_loss()` will require prediction scores from `pipe.predict_proba()`. However, for `roc_auc()` you should only pass the last column `Y_pred_proba[:, 1]`. Use `Y_pred_proba` with `neg_log_loss()`.

In [26]:
model_pipeline.fit(X_train, Y_train)
y_pred = model_pipeline.predict(X_test)
Y_pred_proba = model_pipeline.predict_proba(X_test)
test_metrics = {
    'neg_log_loss': log_loss(Y_test, Y_pred_proba),
    'roc_auc': roc_auc_score(Y_test, Y_pred_proba[:, 1]),
    'accuracy': accuracy_score(Y_test, y_pred),
    'balanced_accuracy': balanced_accuracy_score(Y_test, model_pipeline.predict(X_test))
}
test_metrics

{'neg_log_loss': 0.3744421797453472,
 'roc_auc': 0.9019873780278908,
 'accuracy': 0.8551540587572934,
 'balanced_accuracy': 0.7750141587061985}

# Target Recoding

In the first code chunk of this document, we loaded the data and immediately recoded the target variable `income`. Why is this [convenient](https://scikit-learn.org/stable/modules/model_evaluation.html#binary-case)?

The specific line was:

```
adult_dt = (pd.read_csv('../05_src/data/adult/adult.data', header = None, names = columns)
              .assign(income = lambda x: (x.income.str.strip() == '>50K')*1))
```

- Simplify Classification Task
    - By converting income to binary (0 and 1), it makes the classification task easier for machine learning algorithms. For example, converting '>50K' to 1 and other values (like '<50K') to 0 makes it simpler for algorithms to understand and process.

- Ensure Consistency and Clarity
    - Standardizing the target variable format at the beginning of data processing and model training helps prevent errors or inconsistencies in later steps.

- Ease of Understanding
    - When the target variable is binary, the results are easier to understand. For instance, predicting 1 can be directly interpreted as predicting an income of '>50K', while predicting 0 means predicting an income of '<50K'.

- Facilitate Model Evaluation
    - Many scikit-learn model evaluation metrics (such as accuracy, precision, recall, F1 score, ROC AUC, and log loss) typically expect the target variable to be binary. Converting the target variable early makes it easier to use these metrics.

- Improve Data Processing Efficiency
    - Completing the conversion during data loading is more efficient, as it ensures the transformation is only done once, reducing the need for additional transformation steps and maintaining data consistency.

## Criteria

|Criteria|Complete|Incomplete|
|---------------------|----|----|
|Evaluation of model pipeline |Model pipeline was evaluated correctly.|Model pipeline was not evaluated correctly.|
|Explanation of answer|Answer was concise and explained the learner's reasoning in depth.|Answer was not concise and did not explained the learner's reasoning in depth.|

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-2`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_2.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.

# Reference

Becker,Barry and Kohavi,Ronny. (1996). Adult. UCI Machine Learning Repository. https://doi.org/10.24432/C5XW20.